# B2B Courier charges acuuracy analysis

In [31]:
# Importing lisbraries and Loading the Dataset
import pandas as pd 
import plotly.graph_objects as go


In [ ]:
pip install plotly


In [29]:
orderReport = pd.read_csv("Order Report.csv")
skuMaster = pd.read_csv("SKU Master.csv")
pincodeMapping = pd.read_csv("pincodes.csv")
courierInvoice = pd.read_csv("Invoice.csv")
courierCompanyRates = pd.read_csv("Courier Company - Rates.csv")

print("Order Report: ")
print(orderReport.head())
print()
print("Sku Master:")
print(skuMaster.head())
print("\nPincode Mapping: ")
print(pincodeMapping.head())
print("\nCourier Invoice: ")
print(courierInvoice.head())
print("\nCourier Company Rates: ")
print(courierCompanyRates.head(10))


Order Report: 
   ExternOrderNo            SKU  Order Qty  Unnamed: 3  Unnamed: 4
0     2001827036  8904223818706        1.0         NaN         NaN
1     2001827036  8904223819093        1.0         NaN         NaN
2     2001827036  8904223819109        1.0         NaN         NaN
3     2001827036  8904223818430        1.0         NaN         NaN
4     2001827036  8904223819277        1.0         NaN         NaN

Sku Master:
             SKU  Weight (g)  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  8904223815682         210         NaN         NaN         NaN
1  8904223815859         165         NaN         NaN         NaN
2  8904223815866         113         NaN         NaN         NaN
3  8904223815873          65         NaN         NaN         NaN
4  8904223816214         120         NaN         NaN         NaN

Pincode Mapping: 
   Warehouse Pincode  Customer Pincode Zone  Unnamed: 3  Unnamed: 4
0             121003            507101    d         NaN         NaN
1             121003     

# checking for Missing values

In [32]:
print("\nMissing Values in Order report: ")
print(orderReport.isnull().sum())

print("\nMissing Values in SKu master: ")
print(skuMaster.isnull().sum())

print("\nMissing Values in pincode mapping: ")
print(pincodeMapping.isnull().sum())

print("\nMissing Values in Courier Invoice: ")
print(courierInvoice.isnull().sum())

print("\nMissing Values in Courier company rates: ")
print(courierCompanyRates.isnull().sum())



Missing Values in Order report: 
ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64

Missing Values in SKu master: 
SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64

Missing Values in pincode mapping: 
Warehouse Pincode      0
Customer Pincode       0
Zone                   0
Unnamed: 3           124
Unnamed: 4           124
dtype: int64

Missing Values in Courier Invoice: 
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

Missing Values in Courier company rates: 
fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_additional    0
fwd_e_fixed         0
fwd_e_additional    0
rto_a_fixed         0


# Cleaning the data:

In [33]:
#Removing Unnamed columns 
orderReport = orderReport.drop(columns=['Unnamed: 3','Unnamed: 4'])
skuMaster = skuMaster.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'])
pincodeMapping = pincodeMapping.drop(columns=['Unnamed: 3','Unnamed: 4'])

In [34]:
skuMaster = skuMaster.drop(columns= ['Unnamed: 2'])

KeyError: "['Unnamed: 2'] not found in axis"

In [35]:
skuMaster.isnull()
pincodeMapping.isnull().sum()

Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64

In [36]:
# Merging Order report and SKU Master dataset
mergeData = pd.merge(orderReport,skuMaster,on='SKU')
mergeData.head()

,ExternOrderNo,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001827036,8904223819093,1.0,150
2,2001827036,8904223819109,1.0,100
3,2001827036,8904223818430,1.0,165
4,2001827036,8904223819277,1.0,350


In [37]:
# merging courier invoice and pincode mapping 
abc_courier = pincodeMapping.drop_duplicates(subset=['Customer Pincode'])
courier_abc = courierInvoice[['Order ID','Customer Pincode','Type of Shipment']]
pincodes = courier_abc.merge(abc_courier, on='Customer Pincode')
print(pincodes.head())

     Order ID  Customer Pincode Type of Shipment  Warehouse Pincode Zone
0  2001806232            507101  Forward charges             121003    d
1  2001806273            486886  Forward charges             121003    d
2  2001806408            532484  Forward charges             121003    d
3  2001806458            143001  Forward charges             121003    b
4  2001807012            515591  Forward charges             121003    d


In [ ]:
# ExternOrderno. is notngbut order id in other datasets
# renaming "Externorderno." as Orderid in mergeData
mergeData = mergeData.rename(columns={'ExternOrderNo':'Order ID'})
mergeData.head()

In [ ]:
merged2 = mergeData.merge(pincodes,on = 'Order ID')
merged2.head()

In [ ]:
# calculating the weights in KG's
merged2['Weight (kgs)'] = merged2['Weight (g)']/1000
merged2.head()

In [ ]:
def weight_slab(weight):
    i = round(weight%1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5: 
        return int(weight) +1.0
    else:
        return int(weight) + 0.5
merged2["weight slab(KG)"] = merged2['Weight (kgs)'].apply(weight_slab)
courierInvoice['Weight Slab Charged by Courier Company']=(courierInvoice['Charged Weight']).apply(weight_slab)

In [ ]:
courierInvoice = courierInvoice.rename(columns={'Zone':"Delivery Zone charged by company"})
merged2 = merged2.rename(columns = {'Zone':'Delivery Zone as per ABC'})
merged2 = merged2.rename(columns = {'weight slab(KG)':'Weight slab as per ABC'})


In [ ]:
total_expected_charge = []
for _, row in merged2.iterrows():
    fwd_category = 'fwd_'+row['Delivery Zone as per ABC']
    fwd_fixed = courierCompanyRates.at[0, fwd_category + '_fixed']
    fwd_additional = courierCompanyRates.at[0, fwd_category + '_additional']
    rto_category = 'rto_' + row['Delivery Zone as per ABC']
    rto_fixed = courierCompanyRates.at[0, rto_category + '_fixed']
    rto_additional = courierCompanyRates.at[0, rto_category + '_additional']
        
    weight_slab = row['Weight slab as per ABC']

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_expected_charge.append(0)

merged2['Expected Charge as per ABC'] = total_expected_charge
print(merged2.head())

In [ ]:
merged_output = merged2.merge(courierInvoice, on='Order ID')
print(merged_output.head())

In [ ]:
df_diff = merged_output
df_diff['Difference (Rs.)'] = df_diff['Billing Amount (Rs.)'] - df_diff['Expected Charge as per ABC']
df_new = df_diff[['Order ID', 'Difference (Rs.)', 'Expected Charge as per ABC']]

print(df_new.head())

In [ ]:
# Calculate the total orders in each category
total_correctly_charged = len(df_new[df_new['Difference (Rs.)'] == 0])
total_overcharged = len(df_new[df_new['Difference (Rs.)'] > 0])
total_undercharged = len(df_new[df_new['Difference (Rs.)'] < 0])

# Calculate the total amount in each category
amount_overcharged = abs(df_new[df_new['Difference (Rs.)'] > 0]['Difference (Rs.)'].sum())
amount_undercharged = df_new[df_new['Difference (Rs.)'] < 0]['Difference (Rs.)'].sum()
amount_correctly_charged = df_new[df_new['Difference (Rs.)'] == 0]['Expected Charge as per ABC'].sum()

# Create a new DataFrame for the summary
summary_data = {'Description': ['Total Orders where ABC has been correctly charged',
                                'Total Orders where ABC has been overcharged',
                                'Total Orders where ABC has been undercharged'],
                'Count': [total_correctly_charged, total_overcharged, total_undercharged],
                'Amount (Rs.)': [amount_correctly_charged, amount_overcharged, amount_undercharged]}

df_summary = pd.DataFrame(summary_data)

print(df_summary)

In [ ]:

fig = go.Figure(data=go.Pie(labels=df_summary['Description'],
                            values=df_summary['Count'],
                            textinfo='label+percent',
                            hole=0.4))
fig.update_layout(title='Proportion')

fig.show()